In [6]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error
import requests
from config import NEWS_KEY

# Create function that connects to databse. Import into flask file to use this wjere ever a connection needed.
def connector():
    return sqlite3.connect('db/stockdata.db')


In [7]:
def get_articles(menuchoice):
    stock_news = {
    'url': "https://api.newsriver.io/v2/search?query=text%3A",
    'symbol': menuchoice, # this is a text string doesnt have to be only a ticker symbol.see newriver.io docs
    'sort':'discoverDate',
    'sortorder':'DESC',
    'count': '15',
    'header': {"Authorization":NEWS_KEY}
    }

    formatted_url = "{}{}&sortBy={}&sortOrder={}&limit={}".format(stock_news['url'],stock_news["symbol"],stock_news['sort'],stock_news['sortorder'],stock_news['count'])

    response = requests.get(formatted_url, headers=stock_news['header']).json()
    return response

In [8]:
articles = get_articles('AAPL')

In [9]:
articles

[{'id': 'fteHrBK2ENbTVjQXlzXvrHt0BMz_-XO77hvMEfYaweS4MZzaIx5O9UpOaC2d9PZPYeP1OdChoN7T2QJ9Pepr6w',
  'publishDate': '2019-07-01T09:00:09',
  'discoverDate': '2019-07-01T11:27:50.152+0000',
  'title': 'Bull of the Day: The Trade Desk (TTD)',
  'language': 'en',
  'text': 'I last profiled The Trade Desk (TTD) on May 20 in our weekly Top Picks video after the company had reported a strong "beat and raise" first quarter and the stock fell back from all-time highs above $230 to an attractive consolidation zone between $195 and $205. \xa0 I was pounding the table then about the true disruption this $10 billion digital advertising David was creating amidst the Goliaths Facebook and Google.\xa0 \xa0 Why? Not just because they are disrupting a $750 billion global advertising market. And not just because they just entered the largest single-language consumer market in the world, China, with over 500 million people using their mobile phones to search, shop, and engage content. \xa0 But precisely b

In [10]:
article_df = pd.DataFrame(articles)
article_df.head()

,discoverDate,elements,highlight,id,language,metadata,publishDate,score,structuredText,text,title,url,website
0,2019-07-01T11:27:50.152+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...","), and soon enough Disney (DIS ) and Apple (<h...",fteHrBK2ENbTVjQXlzXvrHt0BMz_-XO77hvMEfYaweS4MZ...,en,"{'readTime': {'type': 'readTime', 'seconds': 3...",2019-07-01T09:00:09,3.273445,<div> \n <p>I last profiled The Trade Desk (TT...,I last profiled The Trade Desk (TTD) on May 20...,Bull of the Day: The Trade Desk (TTD),https://finance.yahoo.com/news/bull-day-trade-...,"{'name': 'Yahoo! Finance', 'hostName': 'financ..."
1,2019-07-01T09:20:23.940+0000,[],: <highlighted>AAPL</highlighted>) habe knapp ...,2qIycYtNztuLz-opfN6YV17yiBCvq_2OSTLmwDCoh3VDf3...,de,"{'readTime': {'type': 'readTime', 'seconds': 63}}",NaN,8.589848,<div> \n <span>Wien (www.aktiencheck.de) - Zum...,Wien (www.aktiencheck.de) - Zum Wochenausklang...,Deutsche Bank besteht US-Stresstest - Designch...,http://www.aktiencheck.de/exklusiv/Artikel-Deu...,"{'name': 'Aktiencheck.de AG', 'hostName': 'www..."
2,2019-07-01T09:20:18.938+0000,[],", Nasdaq-Symbol: <highlighted>AAPL</highlighte...",AC0bBIg8ygssyTGuo3DoCrcjEgr3JkEfwSPA3XRTeKFgwd...,de,"{'readTime': {'type': 'readTime', 'seconds': 1...",NaN,10.961642,<div> \n <span>Hannover (www.aktiencheck.de) -...,Hannover (www.aktiencheck.de) - Apple-Aktienan...,Apple schwächer als der Markt - Aktienanalyse,http://www.aktiencheck.de/exklusiv/Artikel-App...,"{'name': 'Aktiencheck.de AG', 'hostName': 'www..."
3,2019-07-01T07:26:59.173+0000,[],: <highlighted>AAPL</highlighted>) diese gegen...,n7zR43B1GN4tDHGtauRxXIngVcN8-kfKzz4_XPkEn9Sryf...,de,"{'readTime': {'type': 'readTime', 'seconds': 1...",NaN,9.307782,<div> \n <span>München (www.aktiencheck.de) - ...,München (www.aktiencheck.de) - Apple: Die Spek...,Apple: Die Spekulationen haben ein Ende - Über...,http://www.aktiencheck.de/exklusiv/Artikel-App...,"{'name': 'Aktiencheck.de AG', 'hostName': 'www..."
4,2019-07-01T06:29:32.279+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...",iPhone maker’s stock. Several other research ...,fneQDhfCMBnBCHf8r-5h5xY0fTg0Mks99qLgUW4Kl18Vzf...,en,"{'readTime': {'type': 'readTime', 'seconds': 2...",2019-07-01T06:24:43,4.903382,"<div> \n <p><a href=""https://www.marketbeat.c...",Wedbush reiterated their outperform rating on ...,Apple’s (AAPL) “Outperform” Rating Reiterated ...,https://www.themarketsdaily.com/2019/07/01/app...,"{'name': 'themarketsdaily.com/', 'hostName': '..."


In [11]:
article_df = article_df.dropna(subset=['publishDate'])

In [12]:
article_df

,discoverDate,elements,highlight,id,language,metadata,publishDate,score,structuredText,text,title,url,website
0,2019-07-01T11:27:50.152+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...","), and soon enough Disney (DIS ) and Apple (<h...",fteHrBK2ENbTVjQXlzXvrHt0BMz_-XO77hvMEfYaweS4MZ...,en,"{'readTime': {'type': 'readTime', 'seconds': 3...",2019-07-01T09:00:09,3.273445,<div> \n <p>I last profiled The Trade Desk (TT...,I last profiled The Trade Desk (TTD) on May 20...,Bull of the Day: The Trade Desk (TTD),https://finance.yahoo.com/news/bull-day-trade-...,"{'name': 'Yahoo! Finance', 'hostName': 'financ..."
4,2019-07-01T06:29:32.279+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...",iPhone maker’s stock. Several other research ...,fneQDhfCMBnBCHf8r-5h5xY0fTg0Mks99qLgUW4Kl18Vzf...,en,"{'readTime': {'type': 'readTime', 'seconds': 2...",2019-07-01T06:24:43,4.903382,"<div> \n <p><a href=""https://www.marketbeat.c...",Wedbush reiterated their outperform rating on ...,Apple’s (AAPL) “Outperform” Rating Reiterated ...,https://www.themarketsdaily.com/2019/07/01/app...,"{'name': 'themarketsdaily.com/', 'hostName': '..."
5,2019-07-01T05:34:09.872+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...","in ten years, sending <highlighted>AAPL</high...",PzCrbIqeOXKE6FHx749aXe-nmcghctjb7Lbhn1Ak90Finw...,en,"{'readTime': {'type': 'readTime', 'seconds': 6...",2013-12-31T13:05:19,4.239585,"<p><a href=""https://cdn.cultofmac.com/wp-conte...","2013 was an enormous year for Apple. Yes, the...",The Biggest Apple Stories Of 2013 [Year In Rev...,https://www.cultofmac.com/257364/entire-year-a...,{'name': 'Cult of Mac - News and opinion about...
7,2019-06-29T16:12:55.515+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...",Inc. (NASDAQ: <highlighted>AAPL</highlighted>...,67HoH0kDMPVLnZ0gNKUYr85iot5mIKGsVJNw5oOMWxshTe...,en,"{'finSentiment': {'type': 'finSentiment', 'sen...",2019-06-29T00:00:00,2.880554,"<div> \n <p>Somehow, the second quarter of 201...","Somehow, the second quarter of 2019 has come t...","Weekly Recap & Major Analyst Calls, Looking Ba...",https://247wallst.com/investing/2019/06/29/wee...,"{'name': '24/7 Wall St.', 'hostName': '247wall..."
8,2019-06-29T03:27:36.074+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...","Wednesday, which pleased Apple (<highlighted>...",q6LMZW4NYUWLjR7UTgaqVUb2EO-5zoKBNXZY6inTAHbWBT...,en,"{'finSentiment': {'type': 'finSentiment', 'sen...",2019-06-28T14:22:35,15.016048,<div> \n <p>US Treasury Secretary Steven Mnuch...,US Treasury Secretary Steven Mnuchin struck a ...,Apple (AAPL): With High Hopes for Trade War Re...,https://news.yahoo.com/apple-aapl-high-hopes-t...,NaN
9,2019-06-28T22:13:35.128+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...",Apple (<highlighted>AAPL</highlighted>) announ...,qrGd40JJvINZuhk5X-7lmXdm69GD2d81tDwxP84TJTRaoq...,en,"{'readTime': {'type': 'readTime', 'seconds': 1...",2019-06-28T22:12:18,10.453597,<div> \n <p>Apple (AAPL) announced yesterday t...,Apple (AAPL) announced yesterday that Jony Ive...,iPhone Designer Jony Ive Leaves Apple (AAPL): ...,https://www.smarteranalyst.com/nasdaq-stocks/a...,"{'name': 'smarteranalyst.com/', 'hostName': 'w..."
11,2019-06-28T21:20:34.247+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...",Shorted: 17.8 million Source: S3 Partners Tic...,2D_s-VTjmiYTi7oH0DWlfw_-RaAbw8PPSFl_72DaXcPWjh...,en,"{'readTime': {'type': 'readTime', 'seconds': 1...",2019-06-28T18:46:38,7.452451,<div> \n <p></p> \n <p> </p> \n <p> Shorts are...,Shorts are in for summer. As the S&P 500 hits ...,These are the top 15 most-shorted companies in...,https://www.businessinsider.sg/short-selling-s...,"{'name': 'businessinsider.sg/', 'hostName': 'w..."
12,2019-06-28T21:14:06.831+0000,"[{'type': 'Image', 'primary': True, 'url': 'ht...","MacDailyNews Take:Yeah, okay. Hey, whatever it...",9ppSh_eEpVG1XKvIJwAK2m0vbqUzM6-5fOhl713oi-ME3D...,en,"{'readTime': {'type': 'readTime', 'seconds': 55}}",2019-06-28T21:01:14,10.687223,"<div> \n <p>MacDailyNews Take:Y

In [13]:
article_df = article_df[['publishDate','discoverDate','highlight','text','url']]

In [14]:
article_df

,publishDate,discoverDate,highlight,text,url
0,2019-07-01T09:00:09,2019-07-01T11:27:50.152+0000,"), and soon enough Disney (DIS ) and Apple (<h...",I last profiled The Trade Desk (TTD) on May 20...,https://finance.yahoo.com/news/bull-day-trade-...
4,2019-07-01T06:24:43,2019-07-01T06:29:32.279+0000,iPhone maker’s stock. Several other research ...,Wedbush reiterated their outperform rating on ...,https://www.themarketsdaily.com/2019/07/01/app...
5,2013-12-31T13:05:19,2019-07-01T05:34:09.872+0000,"in ten years, sending <highlighted>AAPL</high...","2013 was an enormous year for Apple. Yes, the...",https://www.cultofmac.com/257364/entire-year-a...
7,2019-06-29T00:00:00,2019-06-29T16:12:55.515+0000,Inc. (NASDAQ: <highlighted>AAPL</highlighted>...,"Somehow, the second quarter of 2019 has come t...",https://247wallst.com/investing/2019/06/29/wee...
8,2019-06-28T14:22:35,2019-06-29T03:27:36.074+0000,"Wednesday, which pleased Apple (<highlighted>...",US Treasury Secretary Steven Mnuchin struck a ...,https://news.yahoo.com/apple-aapl-high-hopes-t...
9,2019-06-28T22:12:18,2019-06-28T22:13:35.128+0000,Apple (<highlighted>AAPL</highlighted>) announ...,Apple (AAPL) announced yesterday that Jony Ive...,https://www.smarteranalyst.com/nasdaq-stocks/a...
11,2019-06-28T18:46:38,2019-06-28T21:20:34.247+0000,Shorted: 17.8 million Source: S3 Partners Tic...,Shorts are in for summer. As the S&P 500 hits ...,https://www.businessinsider.sg/short-selling-s...
12,2019-06-28T21:01:14,2019-06-28T21:14:06.831+0000,"MacDailyNews Take:Yeah, okay. Hey, whatever it...","MacDailyNews Take:Yeah, okay. Hey, whatever it...",https://macdailynews.com/2019/06/28/lovefrom-t...
13,2019-06-28T20:28:27,2019-06-28T21:01:45.761+0000,Apple (<highlighted>AAPL</highlighted>) shares...,Apple (AAPL) shares slipped on Thursday as Jon...,https://finance.yahoo.com/news/jony-ives-depar...
14,2019-06-28T18:46:38,2019-06-28T19:01:17.304+0000,: <highlighted>AAPL</highlighted> Market cap: ...,Shorts are in for summer. As the S&P 500 hits ...,https://www.businessinsider.com.au/short-selli...


In [15]:
article_df.to_sql('newsdata',connector(), if_exists='replace')

In [16]:

def init_newstable(responseitem):
    response_df= pd.DataFrame(responseitem)
    response_df.to_sql('newsdata',connector(), if_exists='replace')

In [17]:
init_newstable(articles)

InterfaceError: Error binding parameter 2 - probably unsupported type.

In [ ]:
import tradingeconomics as te
te.login('tnhhla2tsp96sbp:vwin74xflo2vfiy')


In [ ]:
data_df = te.getCalendarData(output_type='df')

In [ ]:
data_df

In [ ]:
url= 'https://www.nasdaq.com/markets/most-active.aspx'

In [ ]:
data = pd.read_html(url)

In [ ]:
most_active_sharevol = data[2]
most_declined = data[4]
mostactive_dollarvol = data[5]

In [ ]:
# most_active_sharevol = most_active_sharevol.drop(['Company.1'], axis=1)
mostactive_dollarvol

In [18]:
import datetime

In [19]:
today = datetime.datetime.now()

In [20]:
today

datetime.datetime(2019, 7, 1, 4, 57, 54, 566629)

In [26]:
formatted_date = today.strftime('%b%d.%Y')

In [27]:
formatted_date

'Jul01.2019'

In [29]:
def testfunct(input,number):
    string = input + str(number)
    print(string)


In [30]:
testfunct('sup')

TypeError: testfunct() missing 1 required positional argument: 'number'